# FIFA WORLD CUP PREDICTION 

In [1]:
import pandas as pd
from string import ascii_uppercase as alphabet
import pickle

In [2]:
all_tables = pd.read_html('https://en.wikipedia.org/wiki/2022_FIFA_World_Cup')

In [3]:
all_tables

[    0                                                  1
 0 NaN  This article documents a current FIFA World Cu...,
    كأس العالم لكرة القدم 2022Kaʾs al-ʿālam li-kurat al-qadam 2022  \
 0   .mw-parser-output .script-arabic{font-family:"...               
 1                                  Tournament details               
 2                                        Host country               
 3                                               Dates               
 4                                               Teams               
 5                                            Venue(s)               
 6                               Tournament statistics               
 7                                      Matches played               
 8                                        Goals scored               
 9                                          Attendance               
 10                                      Top scorer(s)               
 11  ← 2018 2026 → All statistics correct a

In [4]:
all_tables[9]
all_tables[16]
all_tables[23]
all_tables[30]
all_tables[37]
all_tables[44]
all_tables[51]
all_tables[58]

,Pos,Teamvte,Pld,W,D,L,GF,GA,GD,Pts,Qualification
0,1,Portugal,3,2,0,1,6,4,+2,6,Advanced to knockout stage
1,2,South Korea,3,1,1,1,4,4,0,4,Advanced to knockout stage
2,3,Uruguay,3,1,1,1,2,2,0,4,NaN
3,4,Ghana,3,1,0,2,5,7,−2,3,NaN


In [5]:
alphabet

'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [6]:
all_tables = pd.read_html('https://en.wikipedia.org/wiki/2022_FIFA_World_Cup')

dict_table = {}
for letter, i in zip(alphabet, range(9,65,7)):
    df=all_tables[i]
    df.rename(columns={df.columns[1]: 'Team'}, inplace=True)
    df.pop('Qualification')
    dict_table[f'Group {letter}'] = df

In [7]:
dict_table.keys()

dict_keys(['Group A', 'Group B', 'Group C', 'Group D', 'Group E', 'Group F', 'Group G', 'Group H'])

In [8]:
dict_table['Group A']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Netherlands,3,2,1,0,5,1,+4,7
1,2,Senegal,3,2,0,1,5,4,+1,6
2,3,Ecuador,3,1,1,1,4,3,+1,4
3,4,Qatar (H),3,0,0,3,1,7,−6,0


In [9]:
with open('dict_table', 'wb') as output:
    pickle.dump(dict_table, output)

In [10]:
years = [1930,1934,1938,1950,1954,1958,1962,1966,1970,1974,1978,1982,1986,1990,1994,1998,2002,2006,2010,2014,2018]

In [11]:
from bs4 import BeautifulSoup
import requests

In [12]:
def get_matches(year):
    web = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'    
    response = requests.get(web)
    content =(response.text)
    soup = BeautifulSoup(content, 'lxml')
    matches = soup.find_all('div', class_='footballbox')
    
    home = []
    score = []
    away = []
    for match in matches: 
        home.append(match.find('th', class_='fhome').get_text())
        score.append(match.find('th', class_='fscore').get_text())
        away.append(match.find('th', class_='faway').get_text())
    
    dict_football = {'home': home, 'score':score, 'away': away}
    df_football = pd.DataFrame(dict_football)
    df_football['year'] = year
    return df_football

fifa = [get_matches(year) for year in years]
df_fifa = pd.concat(fifa, ignore_index=True)
df_fifa.to_csv('fifa_worldcup_historical_data.csv', index = False)

#fixture
df_fixture = get_matches(2022)
df_fixture.to_csv('fifa_worldcup_fixture.csv', index = False)

In [13]:
#Data Cleaning


In [14]:
df_fixture = pd.read_csv('fifa_worldcup_fixture.csv')

In [15]:
df_fixture['home'] = df_fixture['home'].str.strip()
df_fixture['away'] = df_fixture['away'].str.strip()

In [16]:
df_world = pd.read_csv('fifa_worldcup_historical_data.csv')

In [17]:
df_world.sort_values('year', inplace = True)

In [18]:
df_world

,home,score,away,year
0,France,4–1,Mexico,1930
17,Uruguay,4–2,Argentina,1930
16,Uruguay,6–1,Yugoslavia,1930
15,Argentina,6–1,United States,1930
14,Paraguay,1–0,Belgium,1930
...,...,...,...,...
563,Brazil,2–0,Costa Rica,2018
564,Serbia,1–2,Switzerland,2018
565,Serbia,0–2,Brazil,2018
567,Germany,0–1,Mexico,2018


In [19]:
#Deleting match with walk over
delete_index = df_world[df_world['home'].str.contains('Sweden')& df_world['away'].str.contains('Austria')].index

In [20]:
df_world.drop(index=delete_index, inplace=True)

In [21]:
#columns scores with not only digits and "-"-->[^]: Matches charaters not in brackets(regrex)
df_world['score'] = df_world['score'].str.replace('[^\d-]', '-', regex =True)

In [22]:
#Cleaning score and home/away columns 
df_world['home'] = df_world['home'].str.strip()
df_world['away'] = df_world['away'].str.strip()

In [23]:
#Splitting score columns into home and away goals
df_world[['HomeGoals', 'AwayGoals',2,3,4,5,6,7,8,9,10]] = df_world['score'].str.split('-', expand = True)


In [24]:
df_world.drop(['score',2,3,4,5,6,7,8,9,10], axis = 1, inplace = True)

In [25]:
df_world

,home,away,year,HomeGoals,AwayGoals
0,France,Mexico,1930,4,1
17,Uruguay,Argentina,1930,4,2
16,Uruguay,Yugoslavia,1930,6,1
15,Argentina,United States,1930,6,1
14,Paraguay,Belgium,1930,1,0
...,...,...,...,...,...
563,Brazil,Costa Rica,2018,2,0
564,Serbia,Switzerland,2018,1,2
565,Serbia,Brazil,2018,0,2
567,Germany,Mexico,2018,0,1


In [26]:
#rename columns and changing data types 
df_world.rename(columns={'home':'HomeTeam', 'away':'AwayTeam', 'year':'Year'},inplace=True)
df_world = df_world.astype({'HomeGoals': int, 'AwayGoals':int, 'Year':int})

In [27]:
df_world.dtypes

HomeTeam     object
AwayTeam     object
Year          int32
HomeGoals     int32
AwayGoals     int32
dtype: object

In [28]:
#Creating new column "totalgoals"
df_world['TotalGoals'] = df_world['HomeGoals'] + df_world['AwayGoals']

In [29]:
df_world

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
17,Uruguay,Argentina,1930,4,2,6
16,Uruguay,Yugoslavia,1930,6,1,7
15,Argentina,United States,1930,6,1,7
14,Paraguay,Belgium,1930,1,0,1
...,...,...,...,...,...,...
563,Brazil,Costa Rica,2018,2,0,2
564,Serbia,Switzerland,2018,1,2,3
565,Serbia,Brazil,2018,0,2,2
567,Germany,Mexico,2018,0,1,1


In [30]:
#Exporting clean dataframe

df_world.to_csv('cleanFifa_worldcup_matches.csv', index = False)
df_fixture.to_csv('cleanFifa_worlcup_fixture.csv', index =False)

In [31]:
#Extra verifications 

years = [1930,1934,1938,1950,1954,1958,1962,1966,1970,1974,1978,1982,1986,1990,1994,1998,2002,2006,2010,2014,2018]

for year in years:
    print(year, len(df_world[df_world['Year']==year]))

1930 18
1934 17
1938 18
1950 22
1954 26
1958 35
1962 32
1966 32
1970 8
1974 2
1978 2
1982 4
1986 16
1990 16
1994 16
1998 16
2002 64
2006 64
2010 64
2014 64
2018 64


In [32]:
#split df into df_home and df_away
df_home = df_world[['HomeTeam', 'HomeGoals', 'AwayGoals']]
df_away = df_world[['AwayTeam', 'HomeGoals', 'AwayGoals']]

In [33]:
#rename columns
df_home = df_home.rename(columns={'HomeTeam':'Team', 'HomeGoals': 'GoalsScored', 'AwayGoals':'GoalsConceded'})
df_away = df_away.rename(columns={'AwayTeam':'Team', 'HomeGoals': 'GoalsConceded', 'AwayGoals':'GoalsScored'})

In [34]:
#concat df_home and df_away, group by team and calculate the mean
df_team_strength = pd.concat([df_home, df_away], ignore_index = True).groupby('Team').mean()
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Algeria,1.000000,1.285714
Angola,0.333333,0.666667
Argentina,1.653846,1.230769
Australia,1.000000,2.000000
Austria,2.166667,2.166667
...,...,...
United States,1.291667,1.916667
Uruguay,1.818182,1.295455
Wales,0.800000,0.800000


In [35]:
from scipy.stats import poisson 

In [36]:
#Fucntion predict_points
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        #goal_scored * goals_conceded
        lamb_home = df_team_strength.at[home, 'GoalsScored'] * df_team_strength.at[away, 'GoalsConceded']
        lamb_away = df_team_strength.at[away, 'GoalsScored'] * df_team_strength.at[home, 'GoalsConceded']
        prob_home, prob_away, prob_draw = 0,0,0
        for x in range(0,11): #number of goals home team
            for y in range(0,11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x==y:
                    prob_draw += p
                elif x>y: 
                    prob_away += p
                else: 
                    prob_away += p
                    
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return(points_home, points_away)
    else: 
        return(0,0)
    

In [37]:
predict_points('France', 'Australia' )

(0.11574247778703714, 2.765712203385609)

In [38]:
df_fixture_group_48 = df_fixture[:48].copy()

In [39]:
df_fixture_group_48

,home,score,away,year
0,Qatar,0–2,Ecuador,2022
1,Senegal,0–2,Netherlands,2022
2,Qatar,1–3,Senegal,2022
3,Netherlands,1–1,Ecuador,2022
4,Ecuador,1–2,Senegal,2022
5,Netherlands,2–0,Qatar,2022
6,England,6–2,Iran,2022
7,United States,1–1,Wales,2022
8,Wales,0–2,Iran,2022
9,England,0–0,United States,2022


In [40]:
#Splitting fixture into group, knock out, quarter
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi= df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

In [41]:
df_fixture_quarter

,home,score,away,year
56,Winners Match 53,Match 58,Winners Match 54,2022
57,Netherlands,Match 57,Argentina,2022
58,Winners Match 55,Match 60,Winners Match 56,2022
59,England,Match 59,France,2022


In [42]:
dict_table['Group H']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Portugal,3,2,0,1,6,4,+2,6
1,2,South Korea,3,1,1,1,4,4,0,4
2,3,Uruguay,3,1,1,1,2,2,0,4
3,4,Ghana,3,1,0,2,5,7,−2,3


In [43]:
for group in dict_table: 
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home,away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away
        
    dict_table[group]  = dict_table[group].sort_values('Pts', ascending=False). reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)

In [44]:
dict_table['Group C']

,Team,Pts
0,Mexico,10.0
1,Argentina,9.0
2,Saudi Arabia,9.0
3,Poland,7.0


In [45]:
#Knock Out

df_fixture_knockout

,home,score,away,year
48,Netherlands,3–1,United States,2022
49,Argentina,2–1,Australia,2022
50,France,3–1,Poland,2022
51,England,3–0,Senegal,2022
52,Japan,Match 53,Croatia,2022
53,Brazil,Match 54,South Korea,2022
54,Morocco,Match 55,Spain,2022
55,Portugal,Match 56,Switzerland,2022


In [51]:
#Update the knock out fixture with group winner and ruuners up
for group in dict_table: 
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']
    
    df_fixture_knockout.replace({f'Winners {group}': group_winner, f'Runners_up {group}': runners_up}, inplace = True)
    
df_fixture_knockout['winner'] = '?'
df_fixture_knockout

,home,score,away,year,winner
48,Netherlands,3–1,United States,2022.0,?
49,Argentina,2–1,Australia,2022.0,?
50,France,3–1,Poland,2022.0,?
51,England,3–0,Senegal,2022.0,?
52,Japan,Match 53,Croatia,2022.0,?
53,Brazil,Match 54,South Korea,2022.0,?
54,Morocco,Match 55,Spain,2022.0,?
55,Portugal,Match 56,Switzerland,2022.0,?
47,NaN,NaN,NaN,NaN,?


In [53]:
#Create get_winner function 
def get_winner (df_fixture_updated):
    for index,row in df_fixture_updated.iterrows():
        home, away = row['home'], row['away']
        points_home , points_away = predict_points(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away 
        df_fixture_updated.loc[index, 'winner'] = winner
    return df_fixture_updated

In [49]:
get_winner(df_fixture_knockout)

,home,score,away,year,winner
48,Netherlands,3–1,United States,2022.0,?
49,Argentina,2–1,Australia,2022.0,?
50,France,3–1,Poland,2022.0,?
51,England,3–0,Senegal,2022.0,?
52,Japan,Match 53,Croatia,2022.0,?
53,Brazil,Match 54,South Korea,2022.0,?
54,Morocco,Match 55,Spain,2022.0,?
55,Portugal,Match 56,Switzerland,2022.0,?
47,NaN,NaN,NaN,NaN,Portugal


In [54]:
df_fixture_quarter

,home,score,away,year
56,Winners Match 53,Match 58,Winners Match 54,2022
57,Netherlands,Match 57,Argentina,2022
58,Winners Match 55,Match 60,Winners Match 56,2022
59,England,Match 59,France,2022


In [59]:
#create update_table functio 

def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, roww in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1.loc[index, 'score']
        df_fixture_round_2['winner'] = '?'
        return df_fixture_round_2

In [61]:
update_table(df_fixture_knockout, df_fixture_quarter)

,home,score,away,year,winner
56,Winners Match 53,Match 58,Winners Match 54,2022,?
57,Netherlands,Match 57,Argentina,2022,?
58,Winners Match 55,Match 60,Winners Match 56,2022,?
59,England,Match 59,France,2022,?


In [62]:
get_winner(df_fixture_quarter)

,home,score,away,year,winner
56,Winners Match 53,Match 58,Winners Match 54,2022,Winners Match 54
57,Netherlands,Match 57,Argentina,2022,Argentina
58,Winners Match 55,Match 60,Winners Match 56,2022,Winners Match 56
59,England,Match 59,France,2022,France


In [64]:
#Semifinal
update_table(df_fixture_quarter, df_fixture_semi)

,home,score,away,year,winner
60,Winners Match 57,Match 61,Winners Match 58,2022,?
61,Winners Match 59,Match 62,Winners Match 60,2022,?


In [65]:
get_winner(df_fixture_semi)

,home,score,away,year,winner
60,Winners Match 57,Match 61,Winners Match 58,2022,Winners Match 58
61,Winners Match 59,Match 62,Winners Match 60,2022,Winners Match 60


In [66]:
#Final

update_table(df_fixture_semi, df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Winners Match 61,Match 64,Winners Match 62,2022,?


In [67]:
get_winner(df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,Winners Match 61,Match 64,Winners Match 62,2022,Winners Match 62
